<a href="https://colab.research.google.com/github/dnasywa/projectsPortfolio/blob/main/Topic%20Modelling/LDA_Etika_TI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LDA Bahasa Indonesia

Instalasi beberapa library yang diperlukan, untuk NLTK dan GENSIM sudah include di Colab

In [ ]:
pip install --upgrade gensim

In [ ]:
pip install pyldavis==3.2.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyldavis: filename=pyLDAvis-3.2.1-py2.py3-none-any.whl size=136164 sha256=ac6a2a4dac67fd61181a16b703d3ae0684efaececec466ff57c0037609cd3060
  Stored in directory: /root/.cache/pip/wheels/d3/35/9c/c084bb1c3a4b8a7feb32d57c88f572dcf5c3accc949478893f
Successfully built pyldavis


In [ ]:
pip install Sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 2.5 MB/s eta 0:00:00


In [ ]:
pip install swifter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16506 sha256=b033e746ac149edd502ab12ab8c2775f5fab0520f8e5e90e7ce6aede01efedad
  Stored in directory: /root/.cache/pip/wheels/e4/cf/51/0904952972ee2c7aa3709437065278dc534ec1b8d2ad41b443
Successfully built swifter


# Preparing Data

In [ ]:
import pandas as pd
import numpy as np
import nltk

In [ ]:
#import data dari GDrive
#bisa lihat di video ini https://youtu.be/AAwgqf2d82k
! gdown --id 1046yMQ2zE1KcnTgUqmrrGGmLZmqyzwDw #id file di drive, bisa dilihat di link file dengan menu share file

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1046yMQ2zE1KcnTgUqmrrGGmLZmqyzwDw
To: /content/data_berita_new.xlsx
100% 115k/115k [00:00<00:00, 14.7MB/s]


In [ ]:
dataSB = pd.read_excel('/content/data_berita_new.xlsx', sheet_name="Worksheet") #lokasi file

dataSB.head()

,link,title,pubDate,description,thumbnail
0,https://www.antaranews.com/berita/3883251/vin-...,Vin Diesel digugat mantan asisten atas tuduhan...,2023-12-22T07:13:45.000Z,Aktor Vin Diesel digugat oleh mantan asistenny...,https://img.antaranews.com/cache/800x533/2023/...
1,https://www.antaranews.com/video/3883236/perta...,Pertamina buka 48 SPBU modular di jalur tol,2023-12-22T07:13:18.000Z,ANTARA - PT Pertamina membuka 40 SPBU modular ...,https://img.antaranews.com/cache/800x533/2023/...
2,https://www.antaranews.com/berita/3883248/damk...,Damkar Depok paparkan tips aman selama libur N...,2023-12-22T07:12:33.000Z,"Pemerintah Kota (Pemkot) Depok, Jawa Barat (Ja...",https://img.antaranews.com/cache/800x533/2023/...
3,https://www.antaranews.com/berita/3883245/gibr...,Gibran Center: Gibran akan bahas ekonomi digit...,2023-12-22T07:09:44.000Z,"Sekjen&nbsp;Gibran Center, Fathul Nugroho, men...",https://img.antaranews.com/cache/800x533/2023/...
4,https://www.antaranews.com/berita/3883242/ment...,Menteri Bintang: Hari Ibu momentum apresiasi p...,2023-12-22T07:09:18.000Z,Menteri Pemberdayaan Perempuan dan Perlindunga...,https://img.antaranews.com/cache/800x533/2023/...


In [ ]:
# ------ Case Folding --------
# gunakan fungsi Series.str.lower() pada Pandas

dataSB['description'] = dataSB['description'].str.lower()

print('Case Folding Result : \n')
print(dataSB['description'].head(50))

Case Folding Result : 

0     aktor vin diesel digugat oleh mantan asistenny...
1     antara - pt pertamina membuka 40 spbu modular ...
2     pemerintah kota (pemkot) depok, jawa barat (ja...
3     sekjen&nbsp;gibran center, fathul nugroho, men...
4     menteri pemberdayaan perempuan dan perlindunga...
5     menteri koordinator bidang perekonomian airlan...
6     pertumbuhan produk domestik bruto (pdb) riil a...
7     kantor wilayah (kanwil) badan pertanahan nasio...
8     akademi teknik china (chinese academy of engin...
9     di antara sederet tradisi natal bagi sebagian ...
10    antara - direktorat reserse narkoba&nbsp;(ditr...
11    penyidik komisi pemberantasan korupsi (kpk) me...
12    deputi bidang kebijakan pembangunan brin mego ...
13    banyudono (boyolali) hingga karanganom (klaten...
14    sebanyak 420 pemudik di kota batam, kepulauan ...
15    empat gelar juara menjadi kado pengurus besar ...
16    menteri luar negeri ri retno marsudi bertemu m...
17    antara - presiden 

# Tokenizing

Menghapus karakter-karakter yang tidak penting dari text

In [ ]:
import string
import re #regex library

# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

In [ ]:
# ------ Tokenizing ---------

nltk.download('punkt')

def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")

dataSB['description'] = dataSB['description'].apply(remove_tweet_special)

#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

dataSB['description'] = dataSB['description'].apply(remove_number)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

dataSB['description'] = dataSB['description'].apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

dataSB['description'] = dataSB['description'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

dataSB['description'] = dataSB['description'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

dataSB['description'] = dataSB['description'].apply(remove_singl_char)

# NLTK word tokenize
def word_tokenize_wrapper(text):
    return word_tokenize(text)

dataSB['description_tokens'] = dataSB['description'].apply(word_tokenize_wrapper)

print('Tokenizing Result : \n')
print(dataSB['description_tokens'].head(50))

Tokenizing Result : 

0     [aktor, vin, diesel, digugat, oleh, mantan, as...
1     [antara, pt, pertamina, membuka, spbu, modular...
2     [pemerintah, kota, pemkot, depok, jawa, barat,...
3     [sekjennbspgibran, center, fathul, nugroho, me...
4     [menteri, pemberdayaan, perempuan, dan, perlin...
5     [menteri, koordinator, bidang, perekonomian, a...
6     [pertumbuhan, produk, domestik, bruto, pdb, ri...
7     [kantor, wilayah, kanwil, badan, pertanahan, n...
8     [akademi, teknik, china, chinese, academy, of,...
9     [di, antara, sederet, tradisi, natal, bagi, se...
10    [antara, direktorat, reserse, narkobanbspditre...
11    [penyidik, komisi, pemberantasan, korupsi, kpk...
12    [deputi, bidang, kebijakan, pembangunan, brin,...
13    [banyudono, boyolali, hingga, karanganom, klat...
14    [sebanyak, pemudik, di, kota, batam, kepulauan...
15    [empat, gelar, juara, menjadi, kado, pengurus,...
16    [menteri, luar, negeri, ri, retno, marsudi, be...
17    [antara, presiden, j

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# NLTK calc frequency distribution
def freqDist_wrapper(text):
    return FreqDist(text)

dataSB['description_tokens_fdist'] = dataSB['description_tokens'].apply(freqDist_wrapper)

print('Frequency Tokens : \n')
print(dataSB['description_tokens_fdist'].head(50).apply(lambda x : x.most_common()))

Frequency Tokens : 

0     [(kasus, 2), (pada, 2), (aktor, 1), (vin, 1), ...
1     [(spbu, 2), (trans, 2), (antara, 1), (pt, 1), ...
2     [(kebakaran, 2), (pemerintah, 1), (kota, 1), (...
3     [(sekjennbspgibran, 1), (center, 1), (fathul, ...
4     [(perempuan, 2), (peringatan, 2), (hari, 2), (...
5     [(ekonomi, 2), (menteri, 1), (koordinator, 1),...
6     [(pada, 2), (ketiga, 2), (pertumbuhan, 1), (pr...
7     [(tanah, 2), (kantor, 1), (wilayah, 1), (kanwi...
8     [(dan, 2), (akademi, 1), (teknik, 1), (china, ...
9     [(kue, 3), (di, 2), (natal, 2), (dan, 2), (ant...
10    [(antara, 1), (direktorat, 1), (reserse, 1), (...
11    [(korupsi, 2), (dan, 2), (penyidik, 1), (komis...
12    [(ikan, 2), (deputi, 1), (bidang, 1), (kebijak...
13    [(dan, 2), (banyudono, 1), (boyolali, 1), (hin...
14    [(pt, 2), (pelni, 2), (sebanyak, 1), (pemudik,...
15    [(gelar, 2), (indonesia, 2), (empat, 1), (juar...
16    [(menteri, 1), (luar, 1), (negeri, 1), (ri, 1)...
17    [(di, 2), (surat, 2),

# Stopwords
Membersihkan teks dari kata-kata yang termasuk ke dalam stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords

# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')


# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo',
                       'kalo', 'amp', 'biar', 'bikin', 'bilang',
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih',
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya',
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't',
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah', 'bisnis', 'pandemi', 'indonesia'])

# convert list to dictionary
list_stopwords = set(list_stopwords)

#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

dataSB['description_tokens_WSW'] = dataSB['description_tokens'].apply(stopwords_removal)

print(dataSB['description_tokens_WSW'].head(50))

0     [aktor, vin, diesel, digugat, mantan, asistenn...
1     [pt, pertamina, membuka, spbu, modular, tempor...
2     [pemerintah, kota, pemkot, depok, jawa, barat,...
3     [sekjennbspgibran, center, fathul, nugroho, ca...
4     [menteri, pemberdayaan, perempuan, perlindunga...
5     [menteri, koordinator, bidang, perekonomian, a...
6     [pertumbuhan, produk, domestik, bruto, pdb, ri...
7     [kantor, wilayah, kanwil, badan, pertanahan, n...
8     [akademi, teknik, china, chinese, academy, of,...
9     [sederet, tradisi, natal, keluarga, menyajikan...
10    [direktorat, reserse, narkobanbspditresnarkoba...
11    [penyidik, komisi, pemberantasan, korupsi, kpk...
12    [deputi, bidang, kebijakan, pembangunan, brin,...
13    [banyudono, boyolali, karanganom, klaten, kilo...
14    [pemudik, kota, batam, kepulauan, riau, mengik...
15    [gelar, juara, kado, pengurus, persatuan, catu...
16    [menteri, negeri, ri, retno, marsudi, bertemu,...
17    [presiden, joko, widodo, jakarta, jumat, m

# Normalisasi
Mengganti kata-kata tertentu dengan kata lain yang lebih tepat

In [ ]:
#import file xlsx untuk proses normalisasi
! gdown --id 13YTqouJ2SNqoqlEnxShPVlbD6emiYo9G #id file di drive

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=13YTqouJ2SNqoqlEnxShPVlbD6emiYo9G
To: /content/normalisasi_pbn_new.xlsx
100% 8.56k/8.56k [00:00<00:00, 24.9MB/s]


In [ ]:
normalizad_word = pd.read_excel('/content/normalisasi_pbn_new.xlsx') #lokasi file

normalizad_word_dict = {}

for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1]

def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

dataSB['description_normalized'] = dataSB['description_tokens_WSW'].apply(normalized_term)

dataSB['description_normalized'].head(50)

0     [aktor, vin, diesel, digugat, mantan, asistenn...
1     [pt, pertamina, membuka, spbu, modular, tempor...
2     [pemerintah, kota, pemkot, depok, jawa, barat,...
3     [sekjennbspgibran, center, fathul, nugroho, ca...
4     [menteri, pemberdayaan, perempuan, perlindunga...
5     [menteri, koordinator, bidang, perekonomian, a...
6     [pertumbuhan, produk, domestik, bruto, pdb, ri...
7     [kantor, wilayah, kanwil, badan, pertanahan, n...
8     [akademi, teknik, china, chinese, academy, of,...
9     [sederet, tradisi, natal, keluarga, menyajikan...
10    [direktorat, reserse, narkobanbspditresnarkoba...
11    [penyidik, komisi, pemberantasan, korupsi, kpk...
12    [deputi, bidang, kebijakan, pembangunan, brin,...
13    [banyudono, boyolali, karanganom, klaten, kilo...
14    [pemudik, kota, batam, kepulauan, riau, mengik...
15    [gelar, juara, kado, pengurus, persatuan, catu...
16    [menteri, negeri, ri, retno, marsudi, bertemu,...
17    [presiden, joko, widodo, jakarta, jumat, m

In [ ]:
# import Sastrawi package
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter

# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in dataSB['description_normalized']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '

print(len(term_dict))


3391


In [ ]:
for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)

    # untuk melihat hasilnya silahkan jalankan baris di bawah ini
    # print(term,":" ,term_dict[term])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

dataSB['description_tokens_stemmed'] = dataSB['description_normalized'].swifter.apply(get_stemmed_term)

print(dataSB['description_tokens_stemmed'])

Pandas Apply:   0%|          | 0/620 [00:00<?, ?it/s]

0      [aktor, vin, diesel, gugat, mantan, asisten, l...
1      [pt, pertamina, buka, spbu, modular, temporer,...
2      [perintah, kota, pemkot, depok, jawa, barat, j...
3      [sekjennbspgibran, center, fathul, nugroho, ca...
4      [menteri, daya, perempuan, lindung, anak, bint...
                             ...                        
615    [menteri, negeri,  , keluar, nyata, desak,  , ...
616    [ ,  ,  ,  ,  , podium, debat, cawapres, jumat...
617    [ ,  ,  ,  , love,  ,  ,  , korea, selatan, ce...
618    [polisi, ceko, laku, tembak,  ,  , orang,  , p...
619    [calon, wakil, presiden, ri, nomor, urut, mahf...
Name: description_tokens_stemmed, Length: 620, dtype: object


In [ ]:
#stopwords #2

from nltk.corpus import stopwords

# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')


# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(["ada", "tan", "ton", "pt", "komentar", "juta", "unit", "menang", "artikel",
                       "smartphone", "tagar", "sedia", "kaskus", "seksi"])

# convert list to dictionary
list_stopwords = set(list_stopwords)

#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

dataSB['description_tokens_stemmed2'] = dataSB['description_tokens_stemmed'].apply(stopwords_removal)

print(dataSB['description_tokens_stemmed2'].head(50))

0     [aktor, vin, diesel, gugat, mantan, asisten, l...
1     [pertamina, buka, spbu, modular, temporer, jal...
2     [perintah, kota, pemkot, depok, jawa, barat, j...
3     [sekjennbspgibran, center, fathul, nugroho, ca...
4     [menteri, daya, perempuan, lindung, anak, bint...
5     [menteri, koordinator, bidang, ekonomi, airlan...
6     [tumbuh, produk, domestik, bruto, pdb, riil, a...
7     [kantor, wilayah, kanwil, badan, tanah, nasion...
8     [akademi, teknik, china, chinese, academy, of,...
9     [deret, tradisi, natal, keluarga, saji, kue, k...
10    [direktorat, reserse, narkobanbspditresnarkoba...
11    [sidik, komisi, berantas, korupsi, kpk, sita, ...
12    [deputi, bidang, bijak, bangun, brin, mego, pi...
13    [banyudono, boyolali, karanganom, klaten, kilo...
14    [mudik, kota, batam, pulau, riau, program, mud...
15    [gelar, juara, kado, urus, catur, pb, cas, jel...
16    [menteri, negeri, ri, retno, marsudi, temu, me...
17    [presiden, joko, widodo, jakarta, jumat, s

In [ ]:
for i in range(len(dataSB)):
        a=dataSB.iloc[i][6]
        document.append(a)

document[0:5]

['calon', 'wakil', 'presiden', 'ri', 'nomor']

In [ ]:
doc_clean = dataSB['description_tokens_stemmed2']
doc_clean

0      [aktor, vin, diesel, gugat, mantan, asisten, l...
1      [pertamina, buka, spbu, modular, temporer, jal...
2      [perintah, kota, pemkot, depok, jawa, barat, j...
3      [sekjennbspgibran, center, fathul, nugroho, ca...
4      [menteri, daya, perempuan, lindung, anak, bint...
                             ...                        
615    [menteri, negeri,  , nyata, desak,  , gabung, ...
616    [ ,  ,  ,  ,  , podium, debat, cawapres, jumat...
617    [ ,  ,  ,  , love,  ,  ,  , korea, selatan, ce...
618    [polisi, ceko, laku, tembak,  ,  , orang,  , p...
619    [calon, wakil, presiden, ri, nomor, urut, mahf...
Name: description_tokens_stemmed2, Length: 620, dtype: object

# LDA model using gensim
Proses topic modeling dengan LDA gensim

In [ ]:
import gensim
from gensim import corpora

dictionary = corpora.Dictionary(doc_clean)
print(dictionary)

doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

Dictionary<2371 unique tokens: ['aktor', 'asisten', 'asta', 'diesel', 'film']...>


In [ ]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

total_topics = 3 # jumlah topik yang akan di extract
number_words = 10 # jumlah kata per topik

In [ ]:
# Running and Trainign LDA model on the document term matrix.
lda_model = Lda(doc_term_matrix, num_topics=total_topics, id2word = dictionary, passes=50)

lda_model.show_topics(num_topics=total_topics, num_words=number_words)

[(0,
  '0.015*"cawapres" + 0.014*"presiden" + 0.012*"debat" + 0.011*"jakarta" + 0.010*"jokowi" + 0.008*"md" + 0.008*"mahfud" + 0.008*"nomor" + 0.007*"jpnncom" + 0.007*"kpk"'),
 (1,
  '0.007*"menteri" + 0.007*"kota" + 0.007*"natal" + 0.006*"jakarta" + 0.005*"ekonomi" + 0.004*"jpnncom" + 0.004*"timur" + 0.004*"jatimjpnncom" + 0.004*"rupiah" + 0.004*"surabaya"'),
 (2,
  '0.174*" " + 0.006*"jakarta" + 0.005*"jpnncom" + 0.004*"kerja" + 0.004*"natal" + 0.004*"negeri" + 0.004*"milik" + 0.004*"kota" + 0.004*"desember" + 0.004*"orang"')]

In [ ]:
# Word Count of Topic Keywords

from collections import Counter
topics = lda_model.show_topics(formatted=False)
data_flat = [w for w_list in doc_clean for w in w_list]
counter = Counter(data_flat)

out = []
for i, topic in topics:
    for word, weight in topic:
        out.append([word, i , weight, counter[word]])

df_imp_wcount = pd.DataFrame(out, columns=['word', 'topic_id', 'importance', 'word_count'])
print(df_imp_wcount)

            word  topic_id  importance  word_count
0       cawapres         0    0.015074          52
1       presiden         0    0.013955          50
2          debat         0    0.012192          42
3        jakarta         0    0.011395          77
4         jokowi         0    0.009876          34
5             md         0    0.008446          29
6         mahfud         0    0.008445          29
7          nomor         0    0.007575          26
8        jpnncom         0    0.007068          55
9            kpk         0    0.006740          29
10       menteri         1    0.007490          33
11          kota         1    0.007133          36
12         natal         1    0.006834          33
13       jakarta         1    0.006036          77
14       ekonomi         1    0.005414          48
15       jpnncom         1    0.004377          55
16         timur         1    0.004316          12
17  jatimjpnncom         1    0.004281          13
18        rupiah         1    0

In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
#simpan ke google drive
df_imp_wcount.to_csv('df_imp_wcount.csv')
!cp df_imp_wcount.csv "drive/My Drive/Colab Notebooks/LDA Indonesia/"

In [ ]:
#jika simpan ke local drive
#filedisimpan='df_imp_wcount.xlsx'
#df_imp_wcount.to_excel(filedisimpan, index = False, header=True)

In [ ]:
#Dominant topic and its percentage contribution in each topic
def format_topics_sentences(ldamodel=None, corpus=doc_term_matrix, texts=document):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [ ]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=doc_term_matrix, texts=doc_clean)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
print(df_dominant_topic.head(10))

   Document_No  Dominant_Topic  Topic_Perc_Contrib  \
0            0               2              0.9612   
1            1               2              0.9611   
2            2               1              0.9589   
3            3               0              0.9558   
4            4               0              0.9384   
5            5               1              0.9492   
6            6               2              0.9645   
7            7               1              0.9706   
8            8               1              0.9667   
9            9               2              0.9616   

                                            Keywords  \
0   , jakarta, jpnncom, kerja, natal, negeri, mil...   
1   , jakarta, jpnncom, kerja, natal, negeri, mil...   
2  menteri, kota, natal, jakarta, ekonomi, jpnnco...   
3  cawapres, presiden, debat, jakarta, jokowi, md...   
4  cawapres, presiden, debat, jakarta, jokowi, md...   
5  menteri, kota, natal, jakarta, ekonomi, jpnnco...   
6   , jakarta

In [ ]:
#simpan ke google drive
df_dominant_topic.to_csv('df_dominant_topic.csv')
!cp df_dominant_topic.csv "drive/My Drive/Colab Notebooks/LDA Indonesia/"

In [ ]:
# jika simpan ke local drive
# filedisimpan='df_dominant_topic.xlsx'
# df_dominant_topic.to_excel(filedisimpan, index = False, header=True)

In [ ]:
import pyLDAvis.gensim
import pickle
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()

In [ ]:
import os
LDAvis_data_filepath = os.path.join('ldavis_prepared_'+str(total_topics))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
corpus = [dictionary.doc2bow(text) for text in doc_clean]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# proses ini mungkin agak lama
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
pyLDAvis.save_html(LDAvis_prepared, 'drive/My Drive/Colab Notebooks/LDA Indonesia/ldavis_prepared_'+ str(total_topics) +'.html')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# proses ini mungkin agak lama
LDAvis_prepared

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.119707 -0.076626       1        1  36.725690
0     -0.139816 -0.047721       2        1  35.655233
1      0.020110  0.124347       3        1  27.619077, topic_info=            Term        Freq       Total Category  logprob  loglift
2364              480.000000  480.000000  Default  30.0000  30.0000
280     cawapres   40.000000   40.000000  Default  29.0000  29.0000
274     presiden   39.000000   39.000000  Default  28.0000  28.0000
281        debat   33.000000   33.000000  Default  27.0000  27.0000
269       jokowi   26.000000   26.000000  Default  26.0000  26.0000
...          ...         ...         ...      ...      ...      ...
214   masyarakat    6.135895   15.093930   Topic3  -5.8196   0.3865
228           ri    5.889309   13.567793   Topic3  -5.8606   0.4521
20          jawa    5.910248   14.344837   Topic3  -5.8570   0.4000
270        jumat    6.624839   26.598329   Topic3  -5.7429  -0.1034
662       israel    5.134046   10.532704   Topic3  -5.9978   0.5681

[196 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
2364      1  0.999852            
72        3  0.967885   airlangga
557       1  0.867608        alat
31        3  0.812029  antisipasi
434       3  0.903170      arifin
...     ...       ...         ...
116       1  0.614240     wilayah
116       2  0.087749     wilayah
116       3  0.263246     wilayah
1963      3  0.675625       willy
1474      1  0.945244  yogyakarta

[261 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 2])